In [1]:
%config IPCompleter.greedy=True

In [2]:
import cv2
import numpy as np 

import pandas as pd
import tensorflow as tf
from efficientnet.keras import *
from classification_models.keras import Classifiers

import keras
from keras import backend as K
from keras import Input
from keras.models import Model
from keras.utils import *
from keras.layers import *
from keras.losses import categorical_crossentropy

from albumentations import *
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations import functional as F

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold

from tensorflow import set_random_seed
from tqdm import tnrange, tqdm_notebook
import matplotlib.pyplot as plt

set_random_seed(2020)
np.random.seed(2020)

import os
import gc

Using TensorFlow backend.


In [3]:
'''
    TRAINING CONFIG
'''
IMG_SIZE = (64, 64, 1)
BATCH_SIZE = 256
IMAGE_DATA = '../data/train_images_64x64_raw'
data_name = '64x64_mixup'
test_size=0.15

model_name = 'resnet18'
weights = None
pretrained_weights = None

initial_epoch = 0
initial_lr = 0.001
min_lr = 0.00001
MODE = 'training'
no_of_epochs = 100
epochs_per_cycle = 100

df = pd.read_csv('../data/train.csv')

# train_gen = Compose([
#                     ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=20),
#                     Cutout(num_holes=1, 
#                            max_h_size=IMG_SIZE[0] // 2, 
#                            max_w_size=IMG_SIZE[1] // 2),
#             ])
train_gen = None
val_gen = None

CUTMIX = True
MIXUP = False
ALPHA = 0.2

In [4]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def scheduler(epoch, lr):
    return min_lr + (initial_lr - min_lr) * (1 + np.cos(np.pi * (epoch % epochs_per_cycle) / epochs_per_cycle)) / 2

def build_model():
    base_model = 0
    if 'efficientnet' not in model_name:
        M, _ = Classifiers.get(model_name)
        base_model = M(weights=weights, include_top=False, input_shape=IMG_SIZE)
    else:
        base_model = EfficientNetB0(weights=weights, include_top=False, input_shape=IMG_SIZE)
    x = base_model.output
    x = Dropout(0.2)(x)
    x = GlobalAveragePooling2D()(x)
    o1 = Dense(168, activation='softmax', kernel_initializer='he_normal', name='grapheme')(x)
    o2 = Dense(11, activation='softmax', kernel_initializer='he_normal', name='vowel')(x)
    o3 = Dense(7, activation='softmax', kernel_initializer='he_normal', name='consonant')(x)
    model = Model(inputs=[base_model.input], outputs=[o1,o2,o3])
    if pretrained_weights is not None:
        model.load_weights(pretrained_weights)
    model.compile(optimizer=keras.optimizers.Adam(lr=initial_lr), 
              loss='categorical_crossentropy', 
              metrics=[recall])
    return model

In [5]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, list_id, generator=None, batch_size=32, 
                 use_mixup=False, alpha=0.2, 
                 use_cutmix=False):
        self.data = self.build_data(df)
        self.list_id = list_id
        self.generator = generator
        self.batch_size = batch_size
        self.use_mixup = use_mixup
        self.use_cutmix = use_cutmix
        self.alpha = alpha
    
    def build_data(self, df):
        image_id = df['image_id']
        grapheme_root = df['grapheme_root']
        vowel_diacritic = df['vowel_diacritic']
        consonant_diacritic = df['consonant_diacritic']
        data = {}
        for i in range(len(image_id)):
            data[image_id[i]] = [grapheme_root[i], vowel_diacritic[i], consonant_diacritic[i]]
        return data
    
    def get_data_by_id(self, image_id):
        image_path = os.path.join(IMAGE_DATA, image_id + '.png')
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = np.expand_dims(image, axis=2)
        label = self.data[image_id]
        return image, label
    
    def __len__(self):
        return int(np.floor(len(self.list_id) / self.batch_size))
    
    def on_epoch_end(self):
        np.random.shuffle(self.list_id)
    
    def mixup(self, x, y1, y2, y3):
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        x_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)
        x_t, y1_t, y2_t, y3_t = shuffle(x, y1, y2, y3)

        X = x * x_l + x_t * (1 - x_l)
        Y1 = y1 * y_l + y1_t * (1 - y_l)
        Y2 = y2 * y_l + y2_t * (1 - y_l)
        Y3 = y3 * y_l + y3_t * (1 - y_l)
        return X, Y1, Y2, Y3
    
    def cutmix(self, x, Y_1, Y_2, Y_3):
        cut_ratio = np.random.beta(a=1, b=1, size=self.batch_size)
        cut_ratio = np.clip(cut_ratio, 0.2, 0.8)
        label_ratio = cut_ratio.reshape(self.batch_size, 1)
        x_t, y1_t, y2_t, y3_t = shuffle(x, Y_1, Y_2, Y_3)
        
        cut_img = x_t
        X = x
        for i in range(self.batch_size):
            cut_size = int((IMG_SIZE[0]-1) * cut_ratio[i])
            y1 = np.random.randint(0, (IMG_SIZE[0]-1) - cut_size)
            x1 = np.random.randint(0, (IMG_SIZE[0]-1) - cut_size)
            y2 = y1 + cut_size
            x2 = x1 + cut_size
            cut_arr = cut_img[i][y1:y2, x1:x2]
            cutmix_img = x[i]
            cutmix_img[y1:y2, x1:x2] = cut_arr
            X[i] = cutmix_img
            
        Y1 = Y_1 * (1 - (label_ratio ** 2)) + y1_t * (label_ratio ** 2)
        Y2 = Y_2 * (1 - (label_ratio ** 2)) + y2_t * (label_ratio ** 2)
        Y3 = Y_3 * (1 - (label_ratio ** 2)) + y3_t * (label_ratio ** 2)
        return X, Y1, Y2, Y3
        
        
    def __getitem__(self, index):
        list_id = self.list_id[index*self.batch_size:(index+1)*self.batch_size]
        x = np.empty((len(list_id), *IMG_SIZE), dtype=np.uint8)
        y1 = np.zeros((len(list_id), 168), dtype='float32')
        y2 = np.zeros((len(list_id), 11), dtype='float32')
        y3 = np.zeros((len(list_id), 7), dtype='float32')
        for i in range(len(list_id)):
            image_id = list_id[i]
            image, label = self.get_data_by_id(image_id)
            x[i] = image
            y1[i, label[0]] = 1
            y2[i, label[1]] = 1
            y3[i, label[2]] = 1
        
        if self.use_mixup:
            x, y1, y2, y3 = self.mixup(x.astype('float32'), y1, y2, y3)
        
        if self.use_cutmix:
            x, y1, y2, y3 = self.cutmix(x, y1, y2, y3)
            
        if self.generator is not None:
            for i in range(len(list_id)):
                x[i] = self.generator(image=x[i])['image']

        return x, [y1, y2, y3]

In [6]:
list_id = np.asarray(df['image_id'])
train_list, val_list = train_test_split(list_id, test_size=test_size, random_state=2020)

In [7]:
callbacks = [keras.callbacks.LearningRateScheduler(scheduler, verbose=1),
             keras.callbacks.ModelCheckpoint('../data/model_weights/' + model_name + '/' + model_name + '_{val_grapheme_recall:.4f}_' + data_name + '.h5',
                                         monitor='val_grapheme_recall', 
                                         verbose=1, 
                                         save_best_only=True, 
                                         save_weights_only=True, 
                                         mode='max', 
                                         period=1),
            keras.callbacks.CSVLogger('stats/{}_{}.csv'.format(model_name, data_name))]

model = build_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
if MODE == 'training':
    train_datagen = DataGenerator(df, train_list, generator=train_gen, batch_size=BATCH_SIZE, 
                                  use_mixup=MIXUP, alpha=ALPHA,
                                  use_cutmix=CUTMIX)
    valid_datagen = DataGenerator(df, val_list, generator=val_gen, batch_size=BATCH_SIZE)
    model.fit_generator(train_datagen,
                        steps_per_epoch=len(train_datagen),
                        epochs=no_of_epochs,
                        validation_data=valid_datagen,
                        validation_steps=len(valid_datagen),
                        callbacks=callbacks,
                        verbose=1,
                        initial_epoch=initial_epoch)
elif MODE == 'finetune':
    train_datagen = DataGenerator(df, list_id, generator=None, batch_size=BATCH_SIZE)
    model.fit_generator(train_datagen,
                        steps_per_epoch=len(train_datagen),
                        epochs=no_of_epochs)
    model.save_weights('../data/model_weights/' + model_name + '/' + model_name + '_' + data_name + '.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
666/666 [==============================] - 96s 144ms/step - loss: 6.2961 - grapheme_loss: 3.9927 - vowel_loss: 1.4247 - consonant_loss: 0.8787 - grapheme_recall: 0.0077 - vowel_recall: 0.2830 - consonant_recall: 0.5788 - val_loss: 3.0261 - val_grapheme_loss: 2.0826 - val_vowel_loss: 0.5600 - val_consonant_loss: 0.3835 - val_grapheme_recall: 0.1450 - val_vowel_recall: 0.7508 - val_consonant_recall: 0.8621

Epoch 00001: val_grapheme_recall improved from -inf to 0.14503, saving model to ../data/model_weights/resnet18/resnet18_0.1450_64x64_mixup.h5
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009997557473810372.
666/666 [==============================] - 64s 96ms/step - loss: 5.1297 - grapheme_loss: 3.1317 - vowel_loss: 1.2458 - consonant_loss: 0.7522 - grapheme_recall: 0.0797 - vowel_recall: 0.3975 - consonant_recall: 0.6538 - val_l

666/666 [==============================] - 63s 95ms/step - loss: 3.9378 - grapheme_loss: 2.3121 - vowel_loss: 1.0064 - consonant_loss: 0.6192 - grapheme_recall: 0.3110 - vowel_recall: 0.5077 - consonant_recall: 0.7122 - val_loss: 0.6506 - val_grapheme_loss: 0.3944 - val_vowel_loss: 0.1402 - val_consonant_loss: 0.1160 - val_grapheme_recall: 0.8669 - val_vowel_recall: 0.9717 - val_consonant_recall: 0.9735

Epoch 00013: val_grapheme_recall improved from 0.85430 to 0.86685, saving model to ../data/model_weights/resnet18/resnet18_0.8669_64x64_mixup.h5
Epoch 14/100

Epoch 00014: LearningRateScheduler setting learning rate to 0.0009592885397135706.
666/666 [==============================] - 61s 92ms/step - loss: 3.8923 - grapheme_loss: 2.2853 - vowel_loss: 0.9929 - consonant_loss: 0.6141 - grapheme_recall: 0.3180 - vowel_recall: 0.5130 - consonant_recall: 0.7133 - val_loss: 0.8155 - val_grapheme_loss: 0.4699 - val_vowel_loss: 0.1891 - val_consonant_loss: 0.1565 - val_grapheme_recall: 0.8423 -

666/666 [==============================] - 64s 97ms/step - loss: 3.6072 - grapheme_loss: 2.0978 - vowel_loss: 0.9323 - consonant_loss: 0.5770 - grapheme_recall: 0.3591 - vowel_recall: 0.5270 - consonant_recall: 0.7247 - val_loss: 0.5813 - val_grapheme_loss: 0.3280 - val_vowel_loss: 0.1432 - val_consonant_loss: 0.1102 - val_grapheme_recall: 0.9029 - val_vowel_recall: 0.9789 - val_consonant_recall: 0.9795

Epoch 00026: val_grapheme_recall did not improve from 0.90415
Epoch 27/100

Epoch 00027: LearningRateScheduler setting learning rate to 0.0008438508174347009.
666/666 [==============================] - 65s 97ms/step - loss: 3.5831 - grapheme_loss: 2.0808 - vowel_loss: 0.9274 - consonant_loss: 0.5749 - grapheme_recall: 0.3632 - vowel_recall: 0.5303 - consonant_recall: 0.7264 - val_loss: 0.5384 - val_grapheme_loss: 0.3109 - val_vowel_loss: 0.1269 - val_consonant_loss: 0.1006 - val_grapheme_recall: 0.9038 - val_vowel_recall: 0.9780 - val_consonant_recall: 0.9787

Epoch 00027: val_grapheme

666/666 [==============================] - 62s 93ms/step - loss: 3.4313 - grapheme_loss: 1.9796 - vowel_loss: 0.8951 - consonant_loss: 0.5566 - grapheme_recall: 0.3843 - vowel_recall: 0.5384 - consonant_recall: 0.7310 - val_loss: 0.5040 - val_grapheme_loss: 0.2842 - val_vowel_loss: 0.1231 - val_consonant_loss: 0.0967 - val_grapheme_recall: 0.9232 - val_vowel_recall: 0.9829 - val_consonant_recall: 0.9823

Epoch 00040: val_grapheme_recall improved from 0.92021 to 0.92318, saving model to ../data/model_weights/resnet18/resnet18_0.9232_64x64_mixup.h5
Epoch 41/100

Epoch 00041: LearningRateScheduler setting learning rate to 0.000657963412215599.
666/666 [==============================] - 62s 93ms/step - loss: 3.4292 - grapheme_loss: 1.9788 - vowel_loss: 0.8947 - consonant_loss: 0.5557 - grapheme_recall: 0.3823 - vowel_recall: 0.5380 - consonant_recall: 0.7323 - val_loss: 0.5332 - val_grapheme_loss: 0.2983 - val_vowel_loss: 0.1322 - val_consonant_loss: 0.1027 - val_grapheme_recall: 0.9165 - 

526/666 [======================>.......] - ETA: 12s - loss: 3.2983 - grapheme_loss: 1.8894 - vowel_loss: 0.8699 - consonant_loss: 0.5390 - grapheme_recall: 0.4034 - vowel_recall: 0.5469 - consonant_recall: 0.7391